### Conventional Transfer Learning and Simulation Testing ###

This notebook applies conventional transfer learning and simulation testing for the MAXAR Turkey Marash dataset.

In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import h5py

import shutil
from keras.models import load_model
from keras import layers
from keras import models
from keras import optimizers

In [2]:
model1 = load_model('cc_model_with_dropout_4.h5')
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 sequential (Sequential)     (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                  

In [5]:
test_ds = tf.data.Dataset.load('turkey/test_m_1')
# test_ds = tf.data.Dataset.load('xBD_Dataset/xBD_train_1')

In [6]:
history = model1.evaluate(test_ds)

1250/1250 [==============================] - 20s 9ms/step - loss: 0.8127 - precision: 0.0778 - recall: 0.3359 - accuracy: 0.6740


In [8]:
print(history)

[0.3737606704235077, 0.12715713679790497, 0.16122840344905853, 0.8719393610954285]


In [10]:
# test_predictions = model1.predict(test_ds, batch_size = 32)

In [27]:
model1 = load_model('cc_model_with_dropout_4.h5')
# Disassemble layers
layers = [l for l in model1.layers]


# Stack everything back

x = layers[0].output
for i in range(2, len(layers)):
    if i < len(layers)-2:
        layers[i].trainable = False
    else:
        layers[i].trainable = True
    x = layers[i](x)

# Final touch
new_model = tf.keras.Model(inputs=layers[0].input, outputs=x)
new_model.summary()
    
new_model.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.001*0.1),
                 metrics = [tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),'accuracy'])



Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_input (InputLayer  [(None, 100, 100, 3)]    0         
 )                                                               
                                                                 
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0   

In [28]:
train_ds = tf.data.Dataset.load('turkey/train_m_3')
val_ds = tf.data.Dataset.load('turkey/val_m_3')

history1 = new_model.fit(
            train_ds,
            epochs=50,
            ### IMPORTANT: THIS IS CHANGED
            validation_data=val_ds,
#             class_weight={0:1,1:3}
            )
# history2 = new_model.fit(
#             val_ds,
#             epochs=50,
# #             class_weight={0:1,1:3}
#             )

Epoch 1/50
1136/1136 [==============================] - 11s 10ms/step - loss: 0.3850 - precision_7: 0.0968 - recall_7: 0.0871 - accuracy: 0.8729 - val_loss: 0.2880 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00 - val_accuracy: 0.9178
Epoch 2/50
1136/1136 [==============================] - 9s 8ms/step - loss: 0.2661 - precision_7: 0.3000 - recall_7: 0.0022 - accuracy: 0.9262 - val_loss: 0.2710 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00 - val_accuracy: 0.9180
Epoch 3/50
1136/1136 [==============================] - 8s 7ms/step - loss: 0.2558 - precision_7: 0.4286 - recall_7: 0.0101 - accuracy: 0.9261 - val_loss: 0.2622 - val_precision_7: 1.0000 - val_recall_7: 0.0030 - val_accuracy: 0.9185
Epoch 4/50
1136/1136 [==============================] - 9s 8ms/step - loss: 0.2492 - precision_7: 0.4699 - recall_7: 0.0146 - accuracy: 0.9262 - val_loss: 0.2565 - val_precision_7: 1.0000 - val_recall_7: 0.0091 - val_accuracy: 0.9190
Epoch 5/50
1136/1136 [========================

Epoch 36/50
1136/1136 [==============================] - 9s 8ms/step - loss: 0.2042 - precision_7: 0.6605 - recall_7: 0.1462 - accuracy: 0.9316 - val_loss: 0.2307 - val_precision_7: 0.6765 - val_recall_7: 0.0697 - val_accuracy: 0.9212
Epoch 37/50
1136/1136 [==============================] - 9s 8ms/step - loss: 0.2048 - precision_7: 0.6631 - recall_7: 0.1398 - accuracy: 0.9314 - val_loss: 0.2296 - val_precision_7: 0.6571 - val_recall_7: 0.0697 - val_accuracy: 0.9210
Epoch 38/50
1136/1136 [==============================] - 9s 8ms/step - loss: 0.2035 - precision_7: 0.6380 - recall_7: 0.1469 - accuracy: 0.9311 - val_loss: 0.2304 - val_precision_7: 0.6667 - val_recall_7: 0.0667 - val_accuracy: 0.9210
Epoch 39/50
1136/1136 [==============================] - 9s 8ms/step - loss: 0.2017 - precision_7: 0.6597 - recall_7: 0.1536 - accuracy: 0.9319 - val_loss: 0.2309 - val_precision_7: 0.6216 - val_recall_7: 0.0697 - val_accuracy: 0.9205
Epoch 40/50
1136/1136 [==============================] - 9s 

In [29]:
history = new_model.evaluate(test_ds)

1250/1250 [==============================] - 11s 9ms/step - loss: 0.1784 - precision_7: 0.8422 - recall_7: 0.1466 - accuracy: 0.9382


In [ ]:
counter = 0
for element in test_ds:
    preds = model1.predict(element[0])
    for i in range(element[0].shape[0]):
        img = PIL.Image.fromarray(element[0][i,:,:,:].numpy().astype(np.uint8))
        if preds[i] > 0.5:
            dstatus = 'turkey/test_predictions/undamaged/'
        else:
            dstatus = 'turkey/test_predictions/damaged/'
        counter += 1
        img.save(dstatus + str(counter) + '.png')

1/1 [==============================] - 0s 10ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 26ms/step


In [ ]:
counter

In [56]:
counter = 0
for element in test_ds:
    preds = new_model.predict(element[0])
    for i in range(element[0].shape[0]):
        img = PIL.Image.fromarray(element[0][i,:,:,:].numpy().astype(np.uint8))
        if preds[i] > 0.5:
            dstatus = 'xBD_Dataset/test_predictions/damaged/'
        else:
            dstatus = 'xBD_Dataset/test_predictions/undamaged/'
        counter += 1
        img.save(dstatus + str(counter) + '.png')

1/1 [==============================] - 0s 19ms/step


In [13]:
counter

2353

In [69]:
counter = 0
dataset = test_ds.enumerate()
arr = []
for element in dataset.as_numpy_iterator():
    for i in range(len(element[1][1])):
        dmg = element[1][1][i]   
        img = PIL.Image.fromarray(element[1][0][i].astype(np.uint8))
        if dmg == 1:
            dstatus = 'xBD_Dataset/test_truth/damaged/'
        else:
            dstatus = 'xBD_Dataset/test_truth/undamaged/'
        counter += 1
        img.save(dstatus + str(counter) + '.png')